In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

from data_generator import DFIterator
from model_builder import ModelBuilder

In [2]:
COLUMNS = ['ImageFile', 'POS_X', 'POS_Y', 'POS_Z', 'Q_W', 'Q_X', 'Q_Y', 'Q_Z']
IMG_COLUMN = 'ImageFile'
POSITION_COLUMNS = ['POS_X', 'POS_Y', 'POS_Z']
ORIENTATION_COLUMNS = ['Q_W', 'Q_X', 'Q_Y', 'Q_Z']

DIRECTORY = 'camera_relocalization_sample_dataset/images'
DATA_PATH = 'camera_relocalization_sample_dataset/info.csv'
COLUMN_SEP = ','

IMG_SIZE = (256, 256)
IMG_CROP_SIZE = (224, 224)
POSE_REGRESSOR_SIZE = 1024
BETA = 200
BATCH_SIZE = 50
SHUFFLE = True
RANDOM_SEED = 4444

In [3]:
np.random.seed(RANDOM_SEED)
data = pd.read_csv(DATA_PATH, sep=COLUMN_SEP, usecols=COLUMNS)
train_data, test_data = train_test_split(data, test_size=0.1)
print(f'train size: {train_data.shape[0]}, test_size: {test_data.shape[0]}')
train_data.head()

train size: 900, test_size: 100


,POS_X,POS_Y,POS_Z,Q_W,Q_X,Q_Y,Q_Z,ImageFile
435,1.404367,0.621548,-0.038781,0.926828,0.0,-0.0,-0.375486,img_0_0_1542103551932911900.png
26,-2.294151,-2.619398,-0.038781,0.992391,0.0,-0.0,-0.123129,img_0_0_1542099351874224500.png
456,1.038382,0.173967,-0.038781,0.800721,-0.0,0.0,0.599037,img_0_0_1542103707070959200.png
125,0.748456,-0.351675,-0.038781,0.225952,-0.0,0.0,0.974138,img_0_0_1542100318797415700.png
690,1.482945,-1.242870,-0.038781,1.000000,0.0,-0.0,-0.000002,img_0_0_1542105894912803500.png


In [4]:
train_data.describe()

,POS_X,POS_Y,POS_Z,Q_W,Q_X,Q_Y,Q_Z
count,900.000000,900.000000,9.000000e+02,900.000000,900.0,900.0,900.000000
mean,0.040090,-1.369665,-3.878100e-02,0.709123,0.0,0.0,-0.006197
std,1.483758,1.256425,1.596833e-16,0.317153,0.0,0.0,0.630137
min,-3.077480,-3.778143,-3.878100e-02,0.000018,0.0,-0.0,-1.000000
25%,-1.285080,-2.529762,-3.878100e-02,0.474060,0.0,0.0,-0.535099
50%,0.154681,-1.205611,-3.878100e-02,0.839133,0.0,-0.0,0.002253
75%,1.440197,-0.358070,-3.878100e-02,0.985918,0.0,0.0,0.556028
max,2.812647,1.622524,-3.878100e-02,1.000000,0.0,-0.0,1.000000


In [5]:
position_max = train_data[POSITION_COLUMNS+ORIENTATION_COLUMNS].values.max(axis=0)
position_min = train_data[POSITION_COLUMNS+ORIENTATION_COLUMNS].values.min(axis=0)
position_ranges = position_max - position_min
position_ranges

array([5.890127, 5.400667, 0.      , 0.999982, 0.      , 0.      ,
       2.      ])

In [6]:
imagegen = ImageDataGenerator()
traingen = DFIterator(
    directory=DIRECTORY, dataframe=train_data, image_data_generator=imagegen,
    x_col=IMG_COLUMN, y_col=POSITION_COLUMNS+ORIENTATION_COLUMNS,
    target_size=IMG_SIZE, random_crop_shape=IMG_CROP_SIZE,
    class_mode='raw', batch_size=BATCH_SIZE, shuffle=SHUFFLE)

testgen = DFIterator(
    directory=DIRECTORY, dataframe=test_data, image_data_generator=imagegen,
    x_col=IMG_COLUMN, y_col=POSITION_COLUMNS+ORIENTATION_COLUMNS,
    target_size=IMG_SIZE, random_crop_shape=IMG_CROP_SIZE,
    class_mode='raw', batch_size=BATCH_SIZE, shuffle=SHUFFLE)

Found 900 validated image filenames.
Found 100 validated image filenames.


In [7]:
model_builder = ModelBuilder()

In [8]:
model = model_builder.build_model(image_size=IMG_CROP_SIZE, pose_regressor_size=POSE_REGRESSOR_SIZE,
                                  position_ranges=position_ranges)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model_builder.compile_model(model, lr=0.0001, momentum=0.9, beta=50)

In [12]:
model.fit_generator(traingen, epochs=30, validation_data=testgen, )

Epoch 1/30
18/18 [==============================] - 24s 1s/step - loss: 20.2082 - xyz_loss: 2.1371 - wpqr_loss: 0.2189 - wpqr_orientation_angle: nan - val_loss: 54.6711 - val_xyz_loss: 2.3707 - val_wpqr_loss: 0.9089 - val_wpqr_orientation_angle: 94.9152
Epoch 2/30
18/18 [==============================] - 23s 1s/step - loss: 21.1489 - xyz_loss: 2.1354 - wpqr_loss: 0.2477 - wpqr_orientation_angle: 26.7854 - val_loss: 54.5320 - val_xyz_loss: 2.3215 - val_wpqr_loss: 0.9167 - val_wpqr_orientation_angle: 95.8678
Epoch 3/30
18/18 [==============================] - 24s 1s/step - loss: 20.9170 - xyz_loss: 2.1369 - wpqr_loss: 0.2524 - wpqr_orientation_angle: 27.5161 - val_loss: 54.0801 - val_xyz_loss: 2.3469 - val_wpqr_loss: 0.9161 - val_wpqr_orientation_angle: 95.8036
Epoch 4/30
18/18 [==============================] - 24s 1s/step - loss: 18.8879 - xyz_loss: 2.1354 - wpqr_loss: 0.2205 - wpqr_orientation_angle: 23.8939 - val_loss: 52.3743 - val_xyz_loss: 2.3426 - val_wpqr_loss: 0.8904 - val_wpqr

In [11]:
model.save('model.h5py')

In [ ]:
# from model_builder import RangeNorm
# from losses import euclidean, euclidean_normalized, orientation_angle
# model = tf.keras.models.load_model('model.h5py',
#     custom_objects={'RangeNorm': RangeNorm, 'euclidean': euclidean,
#                     'euclidean_normalized': euclidean_normalized,
#                     'orientation_angle': orientation_angle})